In [101]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def check_fingerprint(wb, matrnr):
    ws = wb["1) Bestellungen"]
    if ws["AS1"].value == int(matrnr):
        return "OK"
    else:
        return "NOK"

def check_exercise_1_1(wb):
    ws = wb["1) Bestellungen"]
    points = 0
    # Check if background color is FH-Mint
    if ws["B2"].fill.start_color.index == ws["J2"].fill.start_color.index == "FF00B1AC":
        points += 1
    if ws["A1"].font.size == ws["J1"].font.size == 16:
        points += 1

    # Check if number-format in column B is Prod-00
    if ws["B10"].number_format == r'"Prod-"00':
        points += 1 

    return points

def check_exercise_1_2(wb, wb_val):
    # Check formulas
    ws = wb["1) Bestellungen"]
    points = 0
    # Check formulas
    if ws["E20"].value != None:
        if ws["E20"].value in ["=C20*D20", "=D20*C20"]:
            points += 1
    
    if ws["C10"].value != None:
        formula = ws["C10"].value.split(",")
        if "VLOOKUP" in formula[0] and "B10" in formula[0] and "$" in formula[1]:
            points += 1

    if ws["K6"].value != None:
        formula = ws["K6"].value.split(",")
        if "COUNTIF" in formula[0] and "B" in formula[0] and "J6" in formula[1]:
            points += 1

    if ws["L7"].value != None:
        formula = ws["L7"].value.split(",")
        if "SUMIF" in formula[0] and "B" in formula[0] and "J7" in formula[1] and "D" in formula[2]:
            points += 1

    if ws["P2"].value != None:
        if "SUM(" in ws["P2"].value:
            points += 1
    if ws["P4"].value != None:
        if "SUMPRODUCT" in ws["P4"].value:
            points += 1

    ws_val = wb_val["1) Bestellungen"]
    if ws_val["P2"].value != None and ws_val["P3"].value != None and ws_val["P4"].value != None:
        if ws_val["P2"].value == ws_val["P3"].value == ws_val["P4"].value:
            points += 1
    
    return points
    
def check_exercise_1_3(wb_val):
    # Check if the calcations for Prod-01 in table "Übersicht" are correct
    ws_val = wb_val["1) Bestellungen"]
    points = 0
    num_prod1 = 0
    turnover_prod1 = 0
    for i in range(3, ws_val.max_row + 1):
        cell = "B" + str(i)
        if ws_val[cell].value == 1:
            cell = "D" + str(i)
            if ws_val[cell].value != None:
                num_prod1 += ws_val[cell].value
            cell = "E" + str(i)
            if ws_val[cell].value != None:
                turnover_prod1 += ws_val[cell].value

    if ws_val["L3"].value != None:
        if ws_val["L3"].value == num_prod1:
            points += 2
    if ws_val["M3"].value != None:
        if round(ws_val["M3"].value, 2) == round(turnover_prod1, 2):
            points += 2
            
    return points

def check_exercise_2_1(wb):
    ws = wb["2) Verkäufe"]
    points = 0
    # Check if background color is FH-Mint
    if ws["B2"].fill.start_color.index == ws["J2"].fill.start_color.index == "FF00B1AC":
        points += 1
    if ws["A1"].font.size == ws["O1"].font.size == 16:
        points += 1

    # Check if number-format in column B is Prod-00
    if ws["B10"].number_format == r'"M-"000':
        points += 1 

    return points

def check_exercise_2_2(wb, wb_val):
    # Check formulas
    ws = wb["2) Verkäufe"]
    points = 0
    # Check formulas
    if ws["E20"].value != None:
        if ws["G20"].value in ["=E20*F20", "=F20*E20"]:
            points += 1

    # Check Formula in 2nd table. COUNTIFS used correctly?
    if ws["K6"].value != None:
        formula = ws["K6"].value.split(",")
        if "COUNTIFS" in formula[0] and \
            ("K$2" in formula[1] or "K$2" in formula[3]) and \
            ("$I6" in formula[1] or "$I6" in formula[3]) and \
            ("$" in formula[0] and "$" in formula[2]):
            points += 1

    # Check Formula in 3rd table. SUMIFS used correctly?
    if ws["R7"].value != None:
        formula = ws["R7"].value.split(",")
        if "SUMIFS" in formula[0] and "$G" in formula[0] and \
            ("R$2" in formula[2] or "R$2" in formula[4]) and \
            ("$O7" in formula[2] or "$O7" in formula[4]) and \
            ("$" in formula[1] and "$" in formula[3]):
            points += 1

    # Check Formula in 4th table. SUMIFS used correctly?
    if ws["Z8"].value != None:
        formula = ws["Z8"].value.split(",")
        if "SUMIFS" in formula[0] and "$G" in formula[0] and \
            ("Z$2" in formula[2] or "Z$2" in formula[4] or "Z$2" in formula[6]) and \
            ("$W8" in formula[2] or "$W8" in formula[4] or "$W8" in formula[6]) and \
            ("$V$2" in formula[2] or "$V$2" in formula[4] or "$V$2" in formula[6]) and \
            ("$" in formula[1] and "$" in formula[3]):
            points += 1
    
    return points

def check_exercise_2_3(wb_val):
    # Check if the calcations for M-001, Amsk, Nord are correct
    ws_val = wb_val["2) Verkäufe"]
    l_products = []
    empl = ws_val["V2"].value
    values_correct = True
    for i in range (3, 23):
        l_products.append(ws_val["I" + str(i)].value)
    for ind in range(len(l_products)):
        turnover = 0
        for i in range(3, ws_val.max_row + 1):
            if ws_val["B" + str(i)].value == empl:
                if ws_val["C" + str(i)].value == "NORD":
                    if ws_val["D" + str(i)].value == l_products[ind]:
                        if ws_val["G" + str(i)].value != None:
                            turnover += ws_val["G" + str(i)].value
        if turnover != ws_val["X" + str(3 + ind)].value:
            values_correct = False
    if values_correct:
        return 5
    else:
        return 0
    
def check_workbook(matrnr, wb, wb_val):
    points = [matrnr]
    points.append(check_fingerprint(wb, matrnr))
    points.append(check_exercise_1_1(wb))
    points.append(check_exercise_1_2(wb, wb_val))
    points.append(check_exercise_1_3(wb_val))
    points.append(check_exercise_2_1(wb))
    points.append(check_exercise_2_2(wb, wb_val))
    points.append(check_exercise_2_3(wb_val))
  
    return points



l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)
for point in l_points:
    print(point)
header = ["MatrNr", "Fingerprint", "1 Format", "1 Formel", "1 Berechnung", "2 Format", "2 Formel", "2 Berechnung"] 
common.result_into_file(l_points, header)


[2054917, 'NOK', 2, 6, 4, 3, 4, 5]
[1459993, 'OK', 0, 0, 0, 0, 0, 0]
[2097324, 'OK', 0, 0, 0, 0, 0, 0]
[1574969, 'OK', 0, 0, 0, 0, 0, 0]
